In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv('train.csv')
df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
data = df.values.T
Y = data[0]
X = data[1:]

Y_dev = Y[:1000]
Y_train = Y[1000:]

X_dev = X[:,:1000]
X_train = X[:,1000:]

In [54]:
def init_params():
    W1 = np.random.rand(128, 784)
    b1 = np.random.rand(128, 1)
    W2 = np.random.rand(10,128)
    b2 = np.random.rand(10, 1)
    
    return W1,W2,b1,b2

def ReLU(Z):
    return np.maximum(Z,0)

def soft_max(Z):
    return np.exp(Z) / np.sum(np.exp(Z), axis = 0)
    
def standardize_data(X):
    mean = np.mean(X,axis = 0)
    std = np.std(X,axis = 0)
    return (X - mean)/std
    
def forward_prop(X,W1,W2,b1,b2):
    Z1 = np.dot(W1, X) + b1
    Z1 = standardize_data(Z1)
    A1 = ReLU(Z1)
    Z2 = np.dot(W2, A1) + b2
    #Z2 = standardize_data(Z2)
    A2 = soft_max(Z2)
    
    return Z1,A1,Z2,A2

def one_hot(Y):
    one_hot_Y = np.zeros((10,Y.size))
    one_hot_Y[Y,np.arange(Y.size)] = 1
    return one_hot_Y

def ReLU_deriv(Z):
    return Z > 0

def backward_prop(Z1,A1,Z2,A2,W2,X,Y):
    m = Y.size
    dZ2 = A2 - one_hot(Y)
    dW2 = 1./(m)*np.dot(dZ2,A1.T)
    db2 = 1./m * np.sum(dZ2,axis = 1,keepdims = True)
    dZ1 = np.dot(W2.T,dZ2) * ReLU_deriv(Z1)
    dW1 = 1./ m * np.dot(dZ1,X.T)
    db1 = 1./m * np.sum(dZ1,axis = 1, keepdims = True)
    return dW1,dW2,db1,db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2
    

In [55]:
def get_predictions(A2):
    return np.argmax(A2, axis = 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, W2, b1, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(X,W1,W2,b1,b2)
        dW1, dW2, db1, db2 = backward_prop(Z1,A1,Z2,A2,W2,X,Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [56]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.90, 500)

Iteration:  0
[6 8 6 ... 8 7 8] [1 5 1 ... 7 6 9]
0.08229268292682927
Iteration:  10
[2 0 2 ... 0 6 2] [1 5 1 ... 7 6 9]
0.40046341463414636
Iteration:  20
[1 8 1 ... 7 6 8] [1 5 1 ... 7 6 9]
0.6538780487804878
Iteration:  30
[1 6 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.8242926829268292
Iteration:  40
[1 8 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9031951219512195
Iteration:  50
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9040975609756098
Iteration:  60
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9178780487804878
Iteration:  70
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9261463414634147
Iteration:  80
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9298780487804879
Iteration:  90
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9337073170731708
Iteration:  100
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9370731707317074
Iteration:  110
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9396829268292682
Iteration:  120
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9414146341463414
Iteration:  130
[1 5 1 ... 7 6 9] [1 5 1 ... 7 6 9]
0.9437073170731707
Iteration:  140